# Preparation

In [ ]:
#Importing various modules required for analysis
import xlsxwriter 
import scipy.sparse as sp
import numpy as np
import os
os.getcwd()
import tkinter as tk
from tkinter import filedialog
import pandas as pd

In [ ]:
#importing the excel. The excel shoud have column named 'Tekst'
root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue')
canvas1.pack()
#input training document
def getExcel ():
    global df_train
    
    import_file_path = filedialog.askopenfilename()
    df_train = pd.read_excel (import_file_path)
    #print (df_train)
    
browseButton_Excel = tk.Button(text='Import Excel File', command=getExcel, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=browseButton_Excel)

root.mainloop()


In [ ]:
#check
df_train.head()

In [ ]:
#creating a column named 'Corpus' from the Text
df_train["Corpus"]=df_train["Tekst"]

## Text tokenizing, lemmatizing, stopword removal

In [ ]:
#dropping the nan values from text
df_train = df_train[df_train['Corpus'].notna()]
#dropping the punctuation from text
import string
string.punctuation
def remove_punctuation(txt):
    txt_nopunct ="".join([c for c in txt if c not in string.punctuation])
    return txt_nopunct
df_train['Corpus_clean']=df_train['Corpus'].apply(lambda x: remove_punctuation(x))
#tokenization
import re
def tokenize(txt):
    tokens= re.split('\W+', txt)
    return tokens
df_train['Corpus_clean_tokenized']=df_train['Corpus_clean'].apply(lambda x:tokenize(x.lower()))
#removing the stopwords
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
df_train['Corpus_nostop']=df_train['Corpus_clean_tokenized'].apply(lambda x: [item for item in x if item not in stop_words])
#lemmatization
import nltk
#nltk.download('wordnet')
wn=nltk.WordNetLemmatizer()
def lemmatization(token_text):
    text= [wn.lemmatize(word) for word in token_text]
    return text
df_train['Corpus_lemmatized']=df_train['Corpus_nostop'].apply(lambda x: lemmatization(x))


In [ ]:
#conversion of columns to lists
train_x=df_train['Corpus_lemmatized'].to_list()

#Converting vector values into string values
train_x = [str (item) for item in train_x]

# K-means clustering

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
#create vectorizer usingTfidfVectorizer class to fit and transform the document 
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(train_x)

Defining no of clusters

In [ ]:
#K means clusering algorithm
true_k = 6 #type number of clusters you required
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

In [ ]:
#making a list of labels
clusters = model.labels_.tolist()

In [ ]:
#to get the centroids and features
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [ ]:
#print the centroids into which clusters they belongs
for i in range(true_k):
    print("cluster ", i),
    for ind in order_centroids[i, :20]:
        print(" ", terms[ind])

In [ ]:
#use joblib.dump to pickle the model, once it has converged and to reload the model/reassign the labels as the clusters.
from sklearn.externals import joblib

In [ ]:
#uncomment the below code to save your model 
#joblib.dump(model,  'doc_cluster.pkl')

In [ ]:
#since I've already run my model I am loading from the pickle
model = joblib.load('doc_cluster.pkl')
clusters = model.labels_.tolist()

In [ ]:
df_train['cluster_kmeans'] = clusters

In [ ]:
df_train['cluster_kmeans'].value_counts()

In [ ]:
#give name to the output excel
df_train.to_excel('name to the output excel.xlsx')